# 6章で作成したモデルのテスト実行

## 事前準備

本ノートブックの実行には以下の事前準備が必要です。

- Google Drive の My Drive 配下に以下のディレクトリ構造でファイルを配置します。

```
MyDrive/JPX_competition/
├── Chapter06
│   ├── 20210224_chapter06_tutorial_test_predictor.ipynb <= 本ノートブック download:11
│   └── archive  <= 投稿用パッケージの起点となるフォルダ
│       ├── model
│       │   ├── headline_features
│       │   │   ├── 19.ckpt  <= 6章で作成した LSTM モデルの学習済みパラメータ download:03
│       │   │   └── LSTM_sentiment.pkl  <= 6章で作成したセンチメント download:04
│       │   ├── my_model_label_high_20.pkl  <= 2章で作成した最高値予測モデル download:01
│       │   ├── my_model_label_low_20.pkl  <= 2章で作成した最安値予測モデル download:02
│       ├── src
│       │   ├── module.py  <= 5章/6章のコードをまとめたもの download:13
│       │   └── predictor.py  <= 4章のコードに一部追記してニュースデータを使用して現金比率操作を追記したもの download:14
│       └── requirements.txt
└── data_dir_comp2
     ├── nikkei_article.csv.gz  <= download:05
     ├── stock_fin.csv.gz  <= download:06
     ├── stock_fin_price.csv.gz  <= download:07
     ├── stock_list.csv.gz  <= download:08
     ├── stock_price.csv.gz  <= download:09
     └── tdnet.csv.gz  <= download:10
```


## 実行環境設定

Google Driveをマウントします。

In [ ]:
import sys

if 'google.colab' in sys.modules:
    # Google Drive をマウントします
    from google.colab import drive
    mount_dir = "/content/drive"
    drive.mount(mount_dir)

環境に応じて使用するディレクトリを設定します。配置した.pyファイルをimportできるようにsys.pathに配置先のディレクトリを追加しています。

In [ ]:
if 'google.colab' in sys.modules:
    # Google Colab環境では上記に示したディレクトリ設定を使用します。
    # archiveディレクトリを指定します。
    archive_path = f"{mount_dir}/MyDrive/JPX_competition/Chapter06/archive"
    # 実装例のコードを配置したディレクトリを指定します。
    src_path = f"{mount_dir}/MyDrive/JPX_competition/Chapter06/archive/src"
    # sys.pathを設定
    sys.path.append(src_path)
    # ダウンロードしてきたデータを配置したディレクトリを設定します。
    dataset_dir = f"{mount_dir}/MyDrive/JPX_competition/data_dir_comp2"
    # 2章のモデルを配置したディレクトリを設定します。
    # このディレクトリにBERTの事前学習済みモデルをダウンロードして保存します。
    model_path = f"{mount_dir}/MyDrive/JPX_competition/Chapter06/archive/model"
    # テスト用に出力したポートフォリオを保存するディレクトリを設定します
    output_path = f"{mount_dir}/MyDrive/JPX_competition/Chapter06"
else:
    # archiveディレクトリを指定します。
    archive_path = "archive"
    # 実装例のコードを配置したディレクトリを指定します。
    src_path = "archive/src"
    # sys.pathを設定
    sys.path.append(src_path)
    # ダウンロードしてきたデータを配置したディレクトリを設定します。
    dataset_dir = "/notebook/data_dir_comp2"
    # 2章のモデルを配置したディレクトリを設定します。
    # このディレクトリにBERTの事前学習済みモデルをダウンロードして保存します。
    model_path = "archive/model"
    # テスト用に出力したポートフォリオを保存するディレクトリを設定します
    output_path = "."

必要なライブラリをインストールします。ここでインストールしているライブラリは投稿用にパッケージ化する際に、requiments.txt に記載します。

In [ ]:
# neologdnのためにg++をインストール
! apt-get update
! apt-get install -y --no-install-recommends g++

In [ ]:
# 必要なライブラリをインストール
!pip install -r $archive_path/requirements.txt

ランタイム環境でpredictメソッドが呼ばれるときに渡される inputs パラメーターを実行環境に合わせて作成します。

In [ ]:
# predictメソッドへの入力パラメーターを設定します。
# ランタイム環境での実行時と同一フォーマットにします
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv.gz",
    "stock_price": f"{dataset_dir}/stock_price.csv.gz",
    "stock_fin": f"{dataset_dir}/stock_fin.csv.gz",
    "stock_fin_price": f"{dataset_dir}/stock_fin_price.csv.gz",
    # ニュースデータ
    "tdnet": f"{dataset_dir}/tdnet.csv.gz",
    "disclosureItems": f"{dataset_dir}/disclosureItems.csv.gz",
    "nikkei_article": f"{dataset_dir}/nikkei_article.csv.gz",
    "article": f"{dataset_dir}/article.csv.gz",
    "industry": f"{dataset_dir}/industry.csv.gz",
    "industry2": f"{dataset_dir}/industry2.csv.gz",
    "region": f"{dataset_dir}/region.csv.gz",
    "theme": f"{dataset_dir}/theme.csv.gz",
    # 目的変数データ
    "stock_labels": f"{dataset_dir}/stock_labels.csv.gz",
    # 購入日指定データ
    "purchase_date": f"{dataset_dir}/purchase_date.csv"
}

## BERTの事前学習済みモデルをダウンロード

ランタイム環境ではインターネットにアクセスできないため、BERTの事前学習済みモデルを `{model_path}/transformers` 配下にダウンロードしておきます。

In [ ]:
from module import SentimentGenerator

In [ ]:
SentimentGenerator.load_feature_extractor(model_path, download=True, save_local=True)

In [ ]:
SentimentGenerator.load_bert_tokenizer(model_path, download=True, save_local=True)

BERTの事前学習済みモデルが保存されていることを確認します。

In [ ]:
! ls -lhR $model_path

## ランタイム環境を想定したテスト実行

ランタイム環境で実行されるのと同等の呼び出し方でテスト実行します。

In [ ]:
from predictor import ScoringService

get_modelメソッドを呼び出すことで以下を実施ます。
1. BERTの事前学習済みモデルを読み込み
2. BERTの事前学習済みモデルに使用したTokenizerを読み込み
3. 事前学習済みの最高値・最安値モデルを読み込み

In [ ]:
ScoringService.get_model(model_path)

ランタイム環境と同一のデータセットを使用していないため、ダウンロードしたデータを使用して動作確認するために予測出力対象日 (start_dt) を 2020-12-28 と指定したpurchase_dateファイルを作成します。このコードを実行することで既に purchase_date.csv が存在している場合は上書きされることに注意してください。

In [ ]:
! echo "Purchase Date" > $dataset_dir/purchase_date.csv
! echo "2020-12-28" >> $dataset_dir/purchase_date.csv

予測を実行します。

In [ ]:
ret = ScoringService.predict(inputs)

## 出力の確認

予測出力の実行結果を確認します。確認ポイントは以下になります。

- 出力のフォーマットが規定されているものと一致していること

In [ ]:
print("\n".join(ret.split("\n")[:10]))

## 出力の保存

バックテストを実行して検証するために出力を保存することもできます。保存したファイルを3章に記載されている方法でバックテストすることでモデル出力を評価することができます。

In [ ]:
# 出力を保存
with open(f"{output_path}/chapter06-tutorial-1.csv", mode="w") as f:
    f.write(ret)

## 投稿用パッケージを作成

In [ ]:
import os
import zipfile

# 提出用パッケージ名
package_file = "chapter06-model.zip"
# パッケージファイルパス
package_path = f"{output_path}/{package_file}"

# zipファイルを作成
with zipfile.ZipFile(package_path, "w") as f:
    # requirements.txt を追加
    print(f"[+] add {archive_path}/requirements.txt to requirements.txt")
    f.write(f"{archive_path}/requirements.txt", "requirements.txt")

    # model/配下を追加
    for root, dirs, files in os.walk(model_path):
        for file in files:
            add_path = os.path.join(root, file)
            rel_path = os.path.relpath(
                os.path.join(root, file),
                os.path.join(model_path, '..')
            )
            print(f"[+] add {add_path} to {rel_path}")
            f.write(add_path, rel_path)

    # src/module.py を追加
    print(f"[+] add {src_path}/module.py to src/module.py")
    f.write(f"{src_path}/module.py", "src/module.py")
    # src/predictor.py を追加
    print(f"[+] add {src_path}/predictor.py to src/predictor.py")
    f.write(f"{src_path}/predictor.py", "src/predictor.py")

print(f"[+] please check {package_path}")